In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [2]:
dataf1 = pd.read_csv('/content/ratings.csv')
dataf2 = pd.read_csv('/content/movies.csv')
merged_dataf = dataf1.merge(dataf2, on='movieId')

In [3]:
del dataf1
del dataf2

In [4]:
merged_dataf.head(5)

,userId,movieId,rating,timestamp,title,genres
0,1,169.0,2.5,1.204928e+09,Free Willy 2: The Adventure Home (1995),Adventure|Children|Drama
1,13,169.0,1.0,9.748684e+08,Free Willy 2: The Adventure Home (1995),Adventure|Children|Drama
2,14,169.0,3.0,8.454703e+08,Free Willy 2: The Adventure Home (1995),Adventure|Children|Drama
3,17,169.0,1.0,9.449914e+08,Free Willy 2: The Adventure Home (1995),Adventure|Children|Drama
4,68,169.0,1.0,1.011092e+09,Free Willy 2: The Adventure Home (1995),Adventure|Children|Drama


In [5]:
merged_dataf.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 659829 entries, 0 to 659828
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     659829 non-null  int64  
 1   movieId    659829 non-null  float64
 2   rating     659829 non-null  float64
 3   timestamp  659829 non-null  float64
 4   title      659829 non-null  object 
 5   genres     659829 non-null  object 
dtypes: float64(3), int64(1), object(2)
memory usage: 35.2+ MB


In [6]:

merged_dataf.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
title        0
genres       0
dtype: int64

In [8]:
merged_dataf[merged_dataf.duplicated()]

,userId,movieId,rating,timestamp,title,genres


In [9]:
merged_dataf.duplicated().sum()

0

In [10]:
merged_dataf["userId"].nunique()

7140

In [11]:
merged_dataf["movieId"].nunique()

15777

In [12]:
user_rating_count = merged_dataf["userId"].value_counts()
values = np.array(user_rating_count.values)
sorted_values = np.sort(values)
print(sorted_values)
print(np.median(sorted_values))
s1 = np.percentile(sorted_values,25)
s3 = np.percentile(sorted_values,75)
r = s3 - s1
print (s1)
print (s3)
print (r)
print (np.sum(sorted_values<=5))

[   1    1    1 ... 3539 3813 5117]
28.0
15.0
85.0
70.0
540


In [13]:
 #plt.figure(figsize=(12, 6))
# sns.barplot(x=movie_ratings_count.index, y=movie_ratings_count.values, palette="viridis")
# plt.title("Number of Users Rating Each Movie")
# plt.xticks(rotation=90)  # Rotate x-axis labels for better visibility
# plt.xlabel("Movie Title")
# plt.ylabel("Number of Users")
# plt.show()

In [14]:
#df_genres = merged_df['genres'].str.get_dummies('|')
#df_genres.head(5)

In [15]:
# df_encoded = pd.concat([merged_df, df_genres], axis=1)
# df_encoded.head(5)

In [16]:
# filtered_df = df_encoded[df_encoded['(no genres listed)'] > 0]
# filtered_df.head(5)

In [19]:
# df_clean = df_encoded.drop(filtered_df.index)
# df_clean.head(5)
df_clean = merged_dataf

In [20]:
# drop merged_df , df_genres , df_endoded , filtered_df
del merged_dataf
# del df_genres
# del df_encoded
# del filtered_df

In [21]:
# seperate the year from movie title to new column (year)
# df_clean[['title', 'year']] = df_clean['title'].str.extract(r'(.+) \((\d{4})\)')
# df_clean.head(5)

In [23]:
#Remove users who rated less than 5 movies and movies rated by less than 2 users
# Count the number of ratings per user and movie
user_ratings_count = df_clean['userId'].value_counts()
movie_ratings_count = df_clean['movieId'].value_counts()

# Filter out users who rated less than 5 movies
users_to_keep = user_ratings_count[user_ratings_count >= 35].index
dataf_filtered_users = df_clean[df_clean['userId'].isin(users_to_keep)]

# Filter out movies rated by less than 2 users
movies_to_keep = movie_ratings_count[movie_ratings_count >= 12].index
dataf_filtered = dataf_filtered_users[dataf_filtered_users['movieId'].isin(movies_to_keep)]

# del df_filtered_users , users_to_keep , movies_to_keep , user_ratings_count , movie_ratings_count
del movie_ratings_count
del user_ratings_count
del dataf_filtered_users
del users_to_keep
del movies_to_keep


# df_filtered now contains only the data for users who rated at least 5 movies and movies rated by at least 2 users.
dataf_filtered.head()

,userId,movieId,rating,timestamp,title,genres
1,13,169.0,1.0,9.748684e+08,Free Willy 2: The Adventure Home (1995),Adventure|Children|Drama
2,14,169.0,3.0,8.454703e+08,Free Willy 2: The Adventure Home (1995),Adventure|Children|Drama
3,17,169.0,1.0,9.449914e+08,Free Willy 2: The Adventure Home (1995),Adventure|Children|Drama
4,68,169.0,1.0,1.011092e+09,Free Willy 2: The Adventure Home (1995),Adventure|Children|Drama
5,178,169.0,2.5,1.140216e+09,Free Willy 2: The Adventure Home (1995),Adventure|Children|Drama


In [24]:
final_dataset = dataf_filtered.pivot(index='movieId',columns='userId',values='rating')
final_dataset

userId,4,11,12,13,14,15,17,18,20,21,...,7116,7123,7124,7125,7128,7132,7133,7135,7137,7140
movieId,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,4.0,5.0,NaN,NaN,NaN,...,4.0,4.5,NaN,NaN,NaN,3.5,3.5,4.0,NaN,NaN
2.0,NaN,NaN,NaN,2.0,NaN,NaN,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,5.0,NaN,3.5,NaN,2.5,NaN,NaN
3.0,NaN,1.5,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,3.0,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139385.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139644.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140110.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
final_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 5568 entries, 1.0 to 148626.0
Columns: 3238 entries, 4 to 7140
dtypes: float64(3238)
memory usage: 137.6 MB


In [26]:
final_dataset

userId,4,11,12,13,14,15,17,18,20,21,...,7116,7123,7124,7125,7128,7132,7133,7135,7137,7140
movieId,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,4.0,5.0,NaN,NaN,NaN,...,4.0,4.5,NaN,NaN,NaN,3.5,3.5,4.0,NaN,NaN
2.0,NaN,NaN,NaN,2.0,NaN,NaN,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,5.0,NaN,3.5,NaN,2.5,NaN,NaN
3.0,NaN,1.5,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,3.0,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139385.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139644.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140110.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
final_dataset.fillna(0,inplace=True)
final_dataset.head()

userId,4,11,12,13,14,15,17,18,20,21,...,7116,7123,7124,7125,7128,7132,7133,7135,7137,7140
movieId,,,,,,,,,,,,,,,,,,,,,
1.0,0.0,0.0,0.0,0.0,0.0,4.0,5.0,0.0,0.0,0.0,...,4.0,4.5,0.0,0.0,0.0,3.5,3.5,4.0,0.0,0.0
2.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,3.5,0.0,2.5,0.0,0.0
3.0,0.0,1.5,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5.0,0.0,0.0,0.0,0.0,3.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
csr_data = csr_matrix(final_dataset.values)
final_dataset.reset_index(inplace=True)

In [29]:
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
knn.fit(csr_data)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)